In [1]:
import glob
import os
import json
import csv
import copy
from openff.toolkit.topology import Molecule
#from make_substructure_labelling_database import *

The idea is reading from pairs of residues with caps (in "main chain" form) and detect the correct number of residues

In [60]:
def check_if_subset(sublist, some_set):
    """Check if sublist is a subset of the given set."""
    return set(sublist).issubset(some_set)
# Check if there are subsets in the matching data (repeated/redundant data)
#for residue_info in matching_data:
print(check_if_subset([1,2,3], [4,2,6,2,1,4,2,4,5,2,3,4,6]))

def remove_subsets(some_list):
    """Remove overlapping sublists (subsets) from list."""
    some_set = set(some_list)
    remove_set = set()
    for element in some_set:
        for other_element in some_set:
            if element != other_element and check_if_subset(element, other_element):
                remove_set.add(element)
    return some_set - remove_set
# Check if subsets are removed
example_list = [(1,2,3), (1,2,3,4), (1,2,3,4,5), (1,2,3), (4,5,6), (8,9,10), (2,3,4,5,6), (1,2,3,4,5,6), (7,8,9,10), (8,9,10), (0,1,2,3), (0,1,2)]
print(remove_subsets(example_list))

True
{(1, 2, 3, 4, 5, 6), (0, 1, 2, 3), (7, 8, 9, 10)}


In [4]:
# Reading the library file into a dictionary
library_file = '../amber-ff-porting/single_res_library.json'
with open(library_file, 'r') as libfile:
    library_dictionary = json.load(libfile)

In [5]:
library_dictionary

{'ACE': ['[H:1][C:2]([H:3])([H:4])[C:5]=[O:6]'],
 'PHE': ['[N:1]([H:2])[C@@:3]([H:4])([C:5]([H:6])([H:7])[c:8]1[c:9]([H:10])[c:11]([H:12])[c:13]([H:14])[c:15]([H:16])[c:17]1[H:18])[C:19]([O-:20])=[O:21]',
  '[N+:1]([H:2])([H:3])([H:4])[C@@:5]([H:6])([C:7]([H:8])([H:9])[c:10]1[c:11]([H:12])[c:13]([H:14])[c:15]([H:16])[c:17]([H:18])[c:19]1[H:20])[C:21]=[O:22]',
  '[N:1]([H:2])[C@@:3]([H:4])([C:5]([H:6])([H:7])[c:8]1[c:9]([H:10])[c:11]([H:12])[c:13]([H:14])[c:15]([H:16])[c:17]1[H:18])[C:19]=[O:20]'],
 'NME': ['[N:1]([H:2])[C:3]([H:4])([H:5])[H:6]'],
 'HIP': ['[N:1]([H:2])[C@@:3]([H:4])([C:5]([H:6])([H:7])[C:8]1~[N:9]([H:10])~[C:11]([H:12])~[N:13]([H:14])~[C:15]~1[H:16])[C:17]([O-:18])=[O:19]',
  '[N:1]([H:2])[C@@:3]([H:4])([C:5]([H:6])([H:7])[C:8]1~[N:9]([H:10])~[C:11]([H:12])~[N:13]([H:14])~[C:15]~1[H:16])[C:17]=[O:18]',
  '[N+:1]([H:2])([H:3])([H:4])[C@@:5]([H:6])([C:7]([H:8])([H:9])[C:10]1~[N:11]([H:12])~[C:13]([H:14])~[N:15]([H:16])~[C:17]~1[H:18])[C:19]=[O:20]'],
 'LYN': ['[N:1]([H:2

In [64]:
# Testing with a single residue
test_file = '../amber-ff-porting/NTerminal/MET/MET.sdf'
offmol = Molecule.from_file(test_file)
matching_data = []
for residue_name, smarts_list in library_dictionary.items():
    for smarts in smarts_list:
        matches = [match for smarts in smarts_list for match in offmol.chemical_environment_matches(smarts)]
        #print(residue_name, matches)
#     for smarts in smarts_list:
#         matches = offmol.chemical_environment_matches(smarts)
#         print(matches)
    matches_unique = remove_subsets(set(tuple(sorted(match)) for match in matches))
    matching_data.append(matches_unique)
    print(residue_name, matches_unique)

ACE set()
PHE set()
NME {(19, 20, 21, 22, 23, 24)}
HIP set()
LYN set()
GLY set()
ASH set()
VAL set()
TRP set()
ASN set()
CYS set()
ALA set()
GLU set()
SER set()
PRO set()
THR set()
ARG set()
GLN set()
HIE set()
ILE set()
LEU set()
TYR set()
ASP set()
HID set()
LYS set()
GLH set()
CYX set()
MET {(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18)}


In [47]:
offmol

NGLWidget()

In [66]:
# Testing with a double residue
test_file = '../amber-ff-porting/MainChain/LYN_ARG/LYN_ARG.sdf'
offmol = Molecule.from_file(test_file)
matching_data = []
for residue_name, smarts_list in library_dictionary.items():
    matches = [match for smarts in smarts_list for match in offmol.chemical_environment_matches(smarts)]
    matches_unique = remove_subsets(set(tuple(sorted(match)) for match in matches if match))
    matching_data.append(matches_unique)
    print(residue_name, matches_unique)

ACE {(0, 1, 2, 3, 4, 5)}
PHE set()
NME {(51, 52, 53, 54, 55, 56)}
HIP set()
LYN {(6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26)}
GLY set()
ASH set()
VAL set()
TRP set()
ASN set()
CYS set()
ALA set()
GLU set()
SER set()
PRO set()
THR set()
ARG {(27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50)}
GLN set()
HIE set()
ILE set()
LEU set()
TYR set()
ASP set()
HID set()
LYS set()
GLH set()
CYX set()
MET set()


In [67]:
offmol

NGLWidget()

In [90]:
# Testing with a double residue
test_file = '../amber-ff-porting/MainChain/LYN_ARG/LYN_ARG.sdf'
offmol = Molecule.from_file(test_file)
matching_data = []
for residue_name, smarts_list in library_dictionary.items():
    matches = [match for smarts in smarts_list for match in offmol.chemical_environment_matches(smarts)]
    #print(matches)
    matches_unique = remove_subsets(set(tuple(sorted(match)) for match in matches))
    matching_data.append(matches_unique)
    print(residue_name, matches_unique)

ACE {(0, 1, 2, 3, 4, 5)}
PHE set()
NME {(51, 52, 53, 54, 55, 56)}
HIP set()
LYN {(6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26)}
GLY set()
ASH set()
VAL set()
TRP set()
ASN set()
CYS set()
ALA set()
GLU set()
SER set()
PRO set()
THR set()
ARG {(27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50)}
GLN set()
HIE set()
ILE set()
LEU set()
TYR set()
ASP set()
HID set()
LYS set()
GLH set()
CYX set()
MET set()


In [91]:
offmol

NGLWidget()

In [75]:
def perceive_residues(offmol):
    """Perceive residue information from a cheminformatics molecule."""
    matches_unique_list = []
    for residue_name, smarts_list in library_dictionary.items():
        matches = [match for smarts in smarts_list for match in offmol.chemical_environment_matches(smarts)]
        matches_unique = set(tuple(sorted(match)) for match in matches)
        if matches_unique:
            matches_unique_list.extend(matches_unique)
    matches_unique_set = remove_subsets(matches_unique_list)
    return matches_unique_set

In [76]:
offmol = Molecule.from_file('../amber-ff-porting/MainChain/CYS_CYX/CYS_CYX.sdf')
perceive_residues(offmol)

{(0, 1, 2, 3, 4, 5),
 (6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16),
 (17, 18, 19, 20, 21, 22, 23, 24, 25, 26),
 (27, 28, 29, 30, 31, 32),
 (33, 34, 35, 36, 37, 38),
 (39, 40, 41, 42, 43, 44, 45, 46, 47, 48),
 (49, 50, 51, 52, 53, 54)}

In [78]:
#%%timeit -n1 -r1
# Testing with every pair of residues in the "database"
test_files = glob.glob('../amber-ff-porting/MainChain/*_*/*.sdf')

matching_data_dict = dict()
for test_file in test_files:
    base_filename = os.path.basename(test_file)
    pair_name = os.path.splitext(base_filename)[0]
    offmol = Molecule.from_file(test_file)
    n_atoms = offmol.n_atoms  # get number of atoms
    matches_unique_set = perceive_residues(offmol)
    #matches_unique_set = remove_subsets(matches_unique_list)
    # Verify that all atoms are matched
    try:
        n_matched_atoms = len([item for sublist in matches_unique_set for item in sublist])
        assert n_matched_atoms == n_atoms
    except AssertionError:
        print(pair_name, n_matched_atoms, n_atoms)
    matching_data_dict[pair_name] = matches_unique_set
    #matching_data_dict[pair_name] = len(matches_unique_set)  # If we want len/size information
#matching_data_dict

In [79]:
matching_data_dict['GLN_HIP']

{(0, 1, 2, 3, 4, 5),
 (6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22),
 (23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40),
 (41, 42, 43, 44, 45, 46)}

In [80]:
matching_data_dict['CYX_ALA']

{(0, 1, 2, 3, 4, 5),
 (6, 7, 8, 9, 10, 11, 12, 13, 14, 15),
 (16, 17, 18, 19, 20, 21, 22, 23, 24, 25),
 (26, 27, 28, 29, 30, 31),
 (32, 33, 34, 35, 36, 37),
 (38, 39, 40, 41, 42, 43, 44, 45, 46, 47),
 (48, 49, 50, 51, 52, 53)}

In [87]:
#%%timeit -n1 -r1
# Testing with a big protein sequence
test_file = '../../../data/biopolymer_smarts/T4.sdf'
offmol = Molecule.from_file(test_file)
matching_data = perceive_residues(offmol)
flattened_info = [item for sublist in matching_data for item in sublist]
# Check number of atoms is the same
assert offmol.n_atoms == len(flattened_info)

3min 36s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [64]:
#%%timeit -n1 -r1
# Testing with every pair of residues in the "database"
test_files = glob.glob('MainChain/*_*/*.sdf')

matching_data_dict = dict()
for test_file in test_files:
    base_filename = os.path.basename(test_file)
    pair_name = os.path.splitext(base_filename)[0]
    offmol = Molecule.from_file(test_file)
    n_atoms = offmol.n_atoms  # get number of atoms
    matches_unique_list = []
    for smarts, residue_name in library_dictionary.items():
        matches = offmol.chemical_environment_matches(smarts)
        matches_unique = set(tuple(sorted(match)) for match in matches)
        if matches_unique:
            matches_unique_list.extend(matches_unique)
    # Remove subsets/overlapping parts
    matches_unique_set = remove_subsets(matches_unique_list)
    # Verify that all atoms are matched
    try:
        n_matched_atoms = len([item for sublist in matches_unique_set for item in sublist])
        assert n_matched_atoms == n_atoms
    except AssertionError:
        print(pair_name, n_matched_atoms, n_atoms)
    matching_data_dict[pair_name] = matches_unique_set
    #matching_data_dict[pair_name] = len(matches_unique_set)  # If we want len/size information
#matching_data_dict

LYN_ARG 33 57
GLN_HIP 46 47


In [61]:
for pair_name, res_matches in matching_data_dict.items():
    size = len(res_matches)
    if size not in [4,7]:
        print(pair_name)

LYN_ARG


In [160]:
# Testing with a big protein sequence
test_file = '../data/biopolymer_smarts/T4.sdf'
offmol = Molecule.from_file(test_file)
matching_data = []
for smarts, residue_name in library_dictionary.items():
    matches = offmol.chemical_environment_matches(smarts)
    matches_unique = set(tuple(sorted(match)) for match in matches)
    matching_data.append(matches_unique)
    print(residue_name, matches_unique)

ACE set()
ALA {(662, 663, 664, 665, 666, 667, 668, 669, 670, 671), (1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163), (1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153), (783, 784, 785, 786, 787, 788, 789, 790, 791, 792), (1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569), (2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322), (1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559), (2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054), (2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113), (1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474), (2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556), (672, 673, 674, 675, 676, 677, 678, 679, 680, 681), (1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296), (2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044), (987, 988, 989, 990, 991, 992, 993, 994, 995, 996), (1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758)}
ARG {(1960, 1961, 1962, 1963, 196

In [218]:
atom_list = []
for key, value in matching_data_dict.items():
    atom_list.extend([item for sublist in value for item in sublist])
len(atom_list)

30943

In [ ]:
# Verifying all atoms are matched
for index, element in enumerate(sorted(set([element for sublist in matching_data for item in sublist for element in item]))):
    if index == element:
        print(index, element)